# AXI Full Burst and Base Overlay Modification
The following is an example on how to modify the Base Overlay and add a custom AXI-Full interface. Our custom full burst interface receives a memory mapped input then adds 5 to that number and stores it in the same address.

The Verilog code that performs these operations is as follows:

``` verilog
    reg 	[C_S_AXI_DATA_WIDTH-1:0] local_memory [10:0];
	reg 	[C_S_AXI_DATA_WIDTH-1:0] local_memory_out [10:0];
	
	always @(posedge S_AXI_ACLK)  
		if ( axi_wready && S_AXI_WVALID ) 
			local_memory[axi_awaddr[ADDR_LSB+OPT_MEM_ADDR_BITS:ADDR_LSB]] <= S_AXI_WDATA+5;
			
	always @(posedge S_AXI_ACLK)
	   local_memory_out[axi_araddr[ADDR_LSB+OPT_MEM_ADDR_BITS:ADDR_LSB]] <= local_memory[axi_araddr[ADDR_LSB+OPT_MEM_ADDR_BITS:ADDR_LSB]];
```

which can be found in lines `612` to `620` of the `axi_full_burst_v1_0_S00_AXI.v` file.

Line `603` was also changed in this code in order to send data stored from memory

``` verilog
    axi_rdata <= local_memory_out[axi_araddr[ADDR_LSB+OPT_MEM_ADDR_BITS:ADDR_LSB]];
```

## The Base Overlay
The base overlay includes logic to interface with all external peripheral interfaces on supported development boards. Depending on the target board, this includes GPIO (buttons, switches, LEDs), HDMI in and out ports, audio ports, Pmod interfaces, and headers for Arduino. A Trace Buffer is also included — this component allows signals passing to and from the external pins to be captured for analysis and debugging purposes. The following figure shows a block diagram of the PYNQ-Z2 base overlay and connected interfaces, although is very similar to the PYNQ-Z1.

![Base Overlay](base_overlay.png)


The PYNQ Base Overlay can be obtained using a linux terminal 

```bash
git clone https://github.com/Xilinx/PYNQ.git
```

Since this example was performed using Vivado 2019.1 we need to checkout into the `image_v2.5.1` branch. We can do so by typing:

```bash
cd PYNQ
git checkout image_v2.5.1
```

Now go inside the folder and run make
```bash
cd /boards/Pynq-Z1/base
make
```

This process will take some time, but if everything works well you will have the Vivado base project in which you can add your custom AXI-Full module.

The following figure shows the custom AXI-Full interface (highlighted in orange)added to the base overlay.

![Base Overlay Modified](base_overlay_modified.png)

The memory mapped interface can also be checked by looking at the Address Editor

![Memory Mapping](memory_mapped.png)


## Running the Base Modified Overlay

In the PYNQ-Z1 board upload into `/pynq/overlays/base_modified` both the `bitstream` and `hardware handoff` files generated by Vivado. Then load your overlay

In [1]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/base_modified/base_ver_1.bit')

Check if your overlay was correctly generated

In [2]:
overlay?

You should see information regarding the IP Blocks and Hierarchies used in the base overlay

```bash
IP Blocks
----------
iop_pmoda/mb_bram_ctrl : pynq.overlay.DefaultIP
switches_gpio        : pynq.lib.axigpio.AxiGPIO
btns_gpio            : pynq.lib.axigpio.AxiGPIO
video/hdmi_in/frontend/axi_gpio_hdmiin : pynq.lib.axigpio.AxiGPIO
video/hdmi_out/frontend/hdmi_out_hpd_video : pynq.lib.axigpio.AxiGPIO
rgbleds_gpio         : pynq.lib.axigpio.AxiGPIO
leds_gpio            : pynq.lib.axigpio.AxiGPIO
system_interrupts    : pynq.overlay.DefaultIP
iop_pmodb/mb_bram_ctrl : pynq.overlay.DefaultIP
video/axi_vdma       : pynq.lib.video.dma.AxiVDMA
audio_direct_0       : pynq.lib.audio.AudioDirect
video/hdmi_out/frontend/axi_dynclk : pynq.overlay.DefaultIP
video/hdmi_out/frontend/vtc_out : pynq.overlay.DefaultIP
video/hdmi_in/frontend/vtc_in : pynq.overlay.DefaultIP
video/hdmi_in/pixel_pack : pynq.lib.video.pipeline.PixelPacker
video/hdmi_in/color_convert : pynq.lib.video.pipeline.ColorConverter
video/hdmi_out/color_convert : pynq.lib.video.pipeline.ColorConverter
video/hdmi_out/pixel_unpack : pynq.lib.video.pipeline.PixelPacker
iop_arduino/mb_bram_ctrl : pynq.overlay.DefaultIP
axi_full_burst_0     : pynq.overlay.DefaultIP
trace_analyzer_pmoda/axi_dma_0 : pynq.lib.dma.DMA
trace_analyzer_arduino/axi_dma_0 : pynq.lib.dma.DMA
trace_analyzer_arduino/trace_cntrl_64_0 : pynq.overlay.DefaultIP
trace_analyzer_pmoda/trace_cntrl_32_0 : pynq.overlay.DefaultIP
ps7_0                : pynq.overlay.DefaultIP

Hierarchies
-----------
iop_arduino          : pynq.lib.pynqmicroblaze.pynqmicroblaze.MicroblazeHierarchy
video/hdmi_in/frontend : pynq.lib.video.dvi.HDMIInFrontend
iop_pmoda            : pynq.lib.pynqmicroblaze.pynqmicroblaze.MicroblazeHierarchy
video/hdmi_out       : pynq.lib.video.hierarchies.VideoOut
video                : pynq.lib.video.hierarchies.HDMIWrapper
video/hdmi_in        : pynq.lib.video.hierarchies.VideoIn
video/hdmi_out/frontend : pynq.lib.video.dvi.HDMIOutFrontend
iop_pmodb            : pynq.lib.pynqmicroblaze.pynqmicroblaze.MicroblazeHierarchy
trace_analyzer_pmoda : pynq.overlay.DefaultHierarchy
trace_analyzer_arduino : pynq.overlay.DefaultHierarchy
```

In Python you will access the AXI interface as an object with the following code

In [3]:
axi_interface = overlay.axi_full_burst_0

We wiil read and write data in two different way. The first way will be by accessing one address at a time and the second way will be using a data stream where just the base address is given.

### Passing data one by one
To send data one by one we need to specify an address for each read or write. For reading we use the `read(address)` method, and for writing we use the `write(address, data)` method. When addressing, be careful because the addressing mode is 4 byte mode.

In [4]:
for i in range(0,16*4,4):
    axi_interface.write(i,i)

In [5]:
for i in range(0,16*4,4):
    print("mem[{}] = {}".format(i,axi_interface.read(i)))

mem[0] = 5
mem[4] = 9
mem[8] = 13
mem[12] = 17
mem[16] = 21
mem[20] = 25
mem[24] = 29
mem[28] = 33
mem[32] = 37
mem[36] = 41
mem[40] = 45
mem[44] = 49
mem[48] = 53
mem[52] = 57
mem[56] = 61
mem[60] = 65


### Passing data by stream
To send data by stream we make use of the MMIO class from the PYNQ framework. The MMIO class allows a Python object to access addresses in the system memory mapped. In particular, registers and address space of peripherals in the PL can be accessed. In an overlay, peripherals connected to the AXI General Purpose ports will have their registers or address space mapped into the system memory map. With PYNQ, the register, or address space of an IP can be accessed from Python using the MMIO class.

In [6]:
from pynq import MMIO
import numpy as np

In [7]:
mmio = MMIO(0x7AA00000,0x10000) #(IP_BASE_ADDRESS, ADDRESS_RANGE)

To access the memory mapped data we can use the `array` method. For example, to write 16 different values we could do

In [8]:
mmio.array[0:16] = np.arange(0,16)

And to check that the values were stored

In [11]:
mmio.array.view(dtype='int32')

array([ 5,  6,  7, ..., 18, 19, 20])

Or just a single value

In [12]:
axi_interface.read(0)

5